## データの読み込み

In [1]:
import pandas as pd
import numpy as np

#local 読み込み
dataPath = "datasets/"
titlePath = 'data/titledata/'

## Titleを抽出してtitle_origin.csvに保存

In [2]:
test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

data = pd.concat([train, test], sort=False)#データの統合

data['title'].to_csv(titlePath + 'title_origin.csv')

## 文字列のクリーニング前処理

In [3]:
from mojimoji import zen_to_han
import re

#unicode正規化
import unicodedata
def normalize_unicode(text):
  return unicodedata.normalize('NFKC', text)

#全角を半角に変更
def zen_han(text):
  for i in range(len(text)):
    text = zen_to_han(text)
  return text

#数字を全て'0'に変更
def num_zero(text):
  return re.sub(r'[0-9]+', "0", text)

#アルファベットを小文字に変換
def lower_text(text):
  return text.lower()

#記号,特殊文字を削除(日本語,英語,カタカナ,数字以外削除)
def delete_symbol(text):
  return re.sub('[^0-9a-zA-Zぁ-んァ-ヿｦ-ﾟ一-龥ー。、]', '、',text)

#特定の文字列を変更
A = [['たこ', 'タコ']]
def change_word(text):
  # for w in A:
  #   print(w[0], w[1])
  #   re.sub(w[0], w[1], text)
  return text

#特定の文字列を削除
def clean_stop_word(text):
  return text

#データのクリーン処理
def clean_text(text):
  text = normalize_unicode(text)
  # text = zen_han(text)
  # text = num_zero(text)
  # text = lower_text(text)
  text = delete_symbol(text)
  # text = change_word(text)
  # text = clean_stop_word(text)
  return text

#titleをクリーニング処理
data = pd.read_csv(titlePath+'title_origin.csv', index_col=0)
data['cleaning'] = data['title'].apply(clean_text)
data.to_csv(titlePath+'cleaning.csv')


## 形態素解析

In [3]:
import MeCab

# MeCab による単語への分割関数,(名詞,形容詞,動詞)のみ残す
def MorphologicalAnalysis(text):
  if text is np.nan:#欠損値は欠損値のまま返す
    return np.NaN
  tagger = MeCab.Tagger()
  words = []
  for c in tagger.parse(text).splitlines()[:-1]:
    surface, feature = c.split('\t')
    pos = feature.split(',')[0]
    if pos in ['名詞', '動詞', '形容詞']:
      words.append(surface)
  return ' '.join(words)

#形態素解析結果を'mecab'に代入
data = pd.read_csv(titlePath+'cleaning.csv', index_col=0)
data['mecab'] = data['cleaning'].apply(MorphologicalAnalysis)
data.to_csv(titlePath+'mecab.csv')

## stopワードの削除(未実装)

## 自作辞書による分散表現

In [ ]:
from gensim.models import word2vec

data = pd.read_csv(titlePath+'mecab.csv', index_col=0)

#[[文章の単語], [文章の単語]]を作成
title_dic = []
def make_title_dic(text):
  if text is np.nan:
    return
  title_dic.append(text.split(' '))
data['mecab'].map(make_title_dic)

#学習
model = word2vec.Word2Vec(title_dic,vector_size=200, min_count=5, window=5, epochs=20)

#作成した辞書をcsvに保存して可視化(処理には関係ない)
from gensim import corpora#辞書を作るためのもの
dictionary = corpora.Dictionary(title_dic)#textsをもとに辞書を作成します
dictionary.filter_extremes(no_below=5)#出現文書数が5回以下のものはさようなら
dictionary.save_as_text(titlePath + 'dictionary.csv')#辞書を保存

#学習結果を確認します
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  #実際の分散表現、size次元の配列

# In[4]:
print(len(model.wv.index_to_key)) # 語彙の数
print(model.wv.index_to_key[:10])

### 文章ベクトルを作成して追加

In [7]:
dimension = 200 #次元

#文章の分散表現を求める(単語ベクトルの平均)
def getSentenceVector(text):
  if text is np.nan:
    return np.array([np.nan for _ in range(dimension)])
  L = []
  for w in text.split(' '):
    if w in model.wv.key_to_index:
      L.append(model.wv.get_vector(key=w))
  if len(L)==0:
    return np.array([np.nan for _ in range(dimension)])
  return np.array(L).mean(axis=0)

#分散表現をnp.arrayの形に変換
def makeVectorArrayList(data):
  vec_list = []
  for index, row in data.iterrows():
    vec_list.append(getSentenceVector(row['mecab']))
  return np.array(vec_list)

data = pd.read_csv(titlePath+'mecab.csv', index_col=0)
vec_list_array = makeVectorArrayList(data)
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(titlePath+'title_vec_originalLearned_'+ str(dimension) +'.csv')

## word2vec 学習済みモデル

### モデルの読み込み

In [2]:
import gensim

# chiVeデータのPATH（kv:KeyedVectors）
model_path = './jalang/entity_vector.model.txt'
# モデルの読み込み
model = gensim.models.KeyedVectors.load_word2vec_format(model_path)

In [ ]:
# モデルの読み込み(別の言語データ)
# import gensim
# from pprint import pprint
# # chiVeデータのPATH（kv:KeyedVectors）
# model_path = './jalang/chive-1.2-mc5.kv'
# # モデルの読み込み
# wv = gensim.models.KeyedVectors.load(model_path)
# # 類似度上位10件を取得
# match = wv.most_similar('大学', topn=10)
# # 見やすい形式で表示
# pprint(match)

### 文章分散表現をもとめる

In [3]:
nonDic = []

#文章の分散表現を求める(単語ベクトルの平均)
def getSentenceVector(text):
  if text is np.nan:
    return np.array([np.nan for _ in range(200)])
  L = []
  for w in text.split(' '):
    if w in model.key_to_index:
      L.append(model.get_vector(key=w))
    else:
      nonDic.append(w)
  if len(L)==0:
    return np.array([np.nan for _ in range(200)])
  return np.array(L).mean(axis=0)

#分散表現をnp.arrayの形に変換
def makeVectorArrayList(data):
  vec_list = []
  for index, row in data.iterrows():
    vec_list.append(getSentenceVector(row['mecab']))
  return np.array(vec_list)

data = pd.read_csv(titlePath+'mecab.csv', index_col=0)
vec_list_array = makeVectorArrayList(data)
vec_data = pd.DataFrame(data=vec_list_array, dtype='float')
vec_data.to_csv(titlePath+'title_vec_learnedModel.csv')


In [7]:
print(len(nonDic))

6409


## 機械学習

In [ ]:

Sentence_vec_data = pd.read_csv(titlePath+'sentenceVec.csv', index_col=0)
